In [1]:
import os
# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes as bnb

In [ ]:
# Set up the 4-bit quantization configuration
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
#set group size
group_size = 128

#load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b", token=True)

# Load the model with quantization configuration
model_q = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b", torch_dtype=torch.bfloat16, token=True,
    quantization_config=quantization_config,
    device_map="auto"  # enables automatic device mapping
)

# Example of modifying model layers to include group size setting
for module in model_q.modules():
    if isinstance(module, bnb.nn.Linear4bit):
        module.group_size = group_size

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Test the model with a simple input
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model_q.generate(inputs.input_ids, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Who are you?"
generated_text = generate_text(prompt)
print(generated_text)

What is your group size?

We can accommodate up to 10 people in our 2-bedroom apartment.

What is the minimum age to rent?

The minimum age to rent is 25 years old.

What is


In [7]:
# Save the quantized model
model_q.save_pretrained("gemma-2-9b-int4")
tokenizer.save_pretrained("gemma-2-9b-int4")

('gemma-2-9b-int4/tokenizer_config.json',
 'gemma-2-9b-int4/special_tokens_map.json',
 'gemma-2-9b-int4/tokenizer.model',
 'gemma-2-9b-int4/added_tokens.json',
 'gemma-2-9b-int4/tokenizer.json')